In [31]:
import pandas as pd
import numpy as np
import seaborn as sb
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import nltk
import re
from nltk.corpus import stopwords

In [3]:
data=pd.read_csv(r"C:\Users\mail2\OneDrive\Desktop\Ml_Data\NLP_DATA\twitter.csv")

In [10]:
data["target"] = data["class"].map({0: "Hate Speech", 1: "Hate Speech", 2: "No Hate and Offensive"})
#data.head()

In [26]:
#data.shape

In [8]:
data_hate_speech=data[["tweet","target"]]

In [25]:
#data_hate_speech.head(1)
#sb.countplot(x="class",data=data)
#plt.show()

In [27]:
#sb.countplot(x="target",data=data_hate_speech)
#plt.show()

In [29]:
#data_hate_speech.head()

In [70]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

### raw tweet 

In [62]:
twwet_raw=np.array(data_hate_speech["tweet"])
twwet_raw_token=word_tokenize(twwet_raw[0])
twwet_raw_token_freq=FreqDist(twwet_raw_token)
#twwet_raw_token_freq.plot()
#plt.show()

## Text Cleanup 

In [71]:
import string
stemmer = nltk.SnowballStemmer("english")
stop_words_list=stopwords.words("english")

In [64]:
#len(stop_words_list)

In [65]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stop_words_list]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [66]:
data_hate_speech.head(1)

,tweet,target
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive


In [72]:
data_hate_speech["clean_tweet"]=data_hate_speech["tweet"].apply(clean)

In [74]:
data_hate_speech.head(1)

,tweet,target,clean_tweet
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive,rt mayasolov woman shouldnt complain clean ho...


### tfidf vectorization

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [76]:
x_tweet=np.array(data_hate_speech["clean_tweet"])
y_target=np.array(data_hate_speech["target"])

In [80]:
#len(np.unique(x_tweet))

In [94]:
tf_idf_vector=TfidfVectorizer(use_idf=True,ngram_range=(2,2))
xvector=tf_idf_vector.fit_transform(x_tweet)

In [96]:
#tf_idf_vector.get_feature_names_out()
#tf_idf_vector.get_feature_names()

In [97]:
xvector.shape

(24783, 124694)

### splitting the data

In [98]:
from sklearn.model_selection import train_test_split

In [101]:
xtrain,xtest,ytrain,ytest=train_test_split(xvector,y_target,test_size=.30,random_state=101)

In [102]:
from sklearn.linear_model import LogisticRegression

In [103]:
lr_trainer=LogisticRegression()
lr_learner=lr_trainer.fit(xtrain,ytrain)

### testing 

In [110]:
input_text=["i spend my money how i want bitch its my business "]

In [113]:
model_input=tf_idf_vector.transform(input_text).toarray()

In [120]:
lr_learner.predict(model_input)
lr_learner.predict_proba(model_input)

array([[0.90575048, 0.09424952]])

In [116]:
#acc

In [117]:
YA=ytest
YP=lr_learner.predict(xtest)

In [118]:
from sklearn.metrics import accuracy_score

In [119]:
accuracy_score(YA,YP)*100

83.68527236045729